In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import random
import os
import sys
import time
import csv
import re
import pandas as pd
import scanpy as sc
import numpy as np
import scipy.spatial as scisp
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
import math
import anndata as ad
import igraph as ig
import plotly.graph_objects as go
import scanpy.external as sce
import scipy.sparse as sp
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.metrics import r2_score
from scipy.interpolate import interp1d
import seaborn as sns
import os
from copy import copy
import matplotlib as mpl
import re
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")
sc.settings.verbosity = 2 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=100, facecolor="white", frameon=True, figsize=(5, 9))
sc.settings.n_jobs=20
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)

# Preprocessing spatial data

In [ ]:
adatast = sc.read_h5ad('/workdir/jp2626/chickenheart/jy/data/raw_with_RAL_updated.h5ad')

for i in adatast.obs['sample'].unique():
    if not re.search('_L_2', i):
        print(i)
        
        adata_subset = adatast[adatast.obs['sample'] == i].copy()
        adata_subset.layers['raw_X'] = adata_subset.X
        adata_subset.var_names_make_unique()
        adata_subset.var["mt"] = adata_subset.var_names.str.startswith("MT-")
        sc.pp.calculate_qc_metrics(adata_subset, qc_vars=["mt"], inplace=True)     
        sc.pp.filter_cells(adata_subset, min_counts=100)
        adata_subset.obsm["spatial_distances"] = scisp.distance.squareform(scisp.distance.pdist(adata_subset.obsm['spatial']))
        adata_subset.obs["spatial_neighbors_100_true"] = np.sum(adata_subset.obsm["spatial_distances"] < 100, axis=0)
        adata_subset = adata_subset[adata_subset.obs['spatial_neighbors_100_true'] >= 15, ]
        
        adata_subset.write_h5ad(f'data/raw_ST_{i}.h5ad')

# Smoothie

In [ ]:
import sys
sys.path.append("/workdir/jp2626/Smoothie/src")

from gaussian_smoothing import *
from spatial_correlation import *
from network_analysis import *
from multi_sample_integration import *
from plotting import *

In [ ]:
for i in adatast.obs['sample'].unique():

    # Check if '_L_2' is not in the sample name
    if not re.search('_L_2', i):
        print(i)

        is_all_integer = np.all(np.floor(adata_subset.X.toarray()) == adata_subset.X.toarray())
        print(is_all_integer)

        sc.pp.normalize_total(adata_subset)
        sc.pp.log1p(adata_subset)

        grid_based_or_not = False
        gaussian_sd = 46.37
        min_spots_under_gaussian = 10
        stride = None
        grid_fitting_dist = None
        num_processes = 10
        num_data_splits = None
        sm_adata = run_parallelized_smoothing(adata_subset,
                                               grid_based_or_not,
                                               gaussian_sd,
                                               min_spots_under_gaussian,
                                               stride,
                                               grid_fitting_dist,
                                               num_processes,
                                               num_data_splits)

        # Save the smoothed data
        sm_adata.write_h5ad(f'data/smoothie_ST_{i}.h5ad')
